# Generating Audio with LSTM 

This notebookw will let you download a song from Youtube and model it with an **LSTM**

This models the way each spectral frame follows another and can be used to generate new raw audio



# Initialise the code and libraries

In [ ]:
%tensorflow_version 1.x
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # less warnings ...
import tensorflow as tf
import numpy as np
from IPython.display import clear_output 
import librosa
from IPython.display import Audio, Image

In [ ]:
!git clone https://github.com/ual-cci/music_gen_interaction_RTML.git

In [ ]:
# python libraries
!pip install Pillow numpy opencv-python PyWavelets tqdm slugify
!pip install -U Flask
!pip install lws==1.2.6
!pip install tflearn
!pip install librosa==0.7.2
!pip install numba==0.48
!pip install mock
clear_output()

In [ ]:
%cd /content/music_gen_interaction_RTML

In [ ]:
# get a youtube downloader
!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl

# Pick a song from Youtube

In [42]:
#Add in your own link
!rm full.wav
!rm youtube_audio.m4a
!youtube-dl -ci -f "bestaudio[ext=m4a]" https://www.youtube.com/watch?v=NUnXxh5U25Y -o youtube_audio.m4a
!ffmpeg -i 'youtube_audio.m4a' -ac 2 -f wav full.wav
clear_output()

# Train Model

You will probably need to train for 300 epochs

In [38]:
number_of_epochs = 300 # will take cca 8min
number_of_epochs = 150 # will take cca 4min

!rm sample.wav
# ----[keep the same bellow]-------------------------------------------------------------------
!ffmpeg -ss 60 -i full.wav -t 60 -c copy sample.wav

!rm -rf __music_samples/
!mkdir __music_samples
!mkdir __music_samples/sample/
!mv sample.wav __music_samples/sample/
!rm -rf __saved_models/
!mkdir __saved_models/

# takes time!
!python training_handler.py -target_file __music_samples/sample/ -amount_epochs $number_of_epochs -batch_size 512
clear_output()

# Set up Generation

Generate some audio with the trained model

In [39]:
from server_handler import ServerHandler
import settings
%cd /content/music_gen_interaction_RTML/

from unittest.mock import Mock, MagicMock
args = MagicMock(name='method')
sample_rate = 22050

# keep the same settings as the model you used when training:
args.lstm_layers = 3
args.lstm_units = 128
args.griffin_iterations = 60
args.sample_rate = sample_rate
args.sequence_length = 40
args.async_loading = True
args.amount_epochs = -1
my_settings = settings.Settings(args)
my_settings.print_settings()

generation_handler = ServerHandler(my_settings)
clear_output()

# Generate!
## Parameters

Pick a position in the song and a length 

If you want to download the output, you can find it the Colab side bar <---

Its called "generated_output.wav". Click to download.

In [41]:
position_in_the_song = 0.7 # between 0.0 and 1.0 - corresponds to 0-100%
requested_length = 600 #number of samples to predict ...
                  #600 corresponds to ~13 sec

generation_handler.change_impulse(position_in_the_song)
audio_arr, t_predict, t_reconstruct = generation_handler.generate_audio_sample(requested_length, interactive_i=position_in_the_song)

# clear possible errors ...
#clear_output()

print(audio_arr.shape)
!rm generated_output.wav
out_name = 'generated_output.wav'
librosa.output.write_wav(out_name, audio_arr, sr=sample_rate)
Audio(out_name)

random_index selected as 1836 from interactive_i= 0.7
precut audio.shape (327168,)
audio.shape (306721,)
(306721,)
